# <B> Bioinformatics Algorithms Example

These exercies and examples are based on the book Bioinformatics Algorithms by Philip Compeau.
You can find a free online copy of the book at this website- https://www.bioinformaticsalgorithms.org/

### A Brief Introduction
As our first exercise, we will try to find the most frequently occuring k-mer in a given DNA string. <br>
The larger idea is that regulatory sequences in DNA usually occur frequently within smaller substrings or lengths of DNA. This helps in identifying the regulatory regions and also important motifs within an organism's DNA. <br>
It is now a well established fact that DNA, the genetic material in most organisms needs to replicate. This replication needs to begin somewhere. This region is called ori (short for origin) and characterized by the occurence of multiple repeats of patterns within a short length. This is essentially what k-mers are- a substring of length k within a biological sequence. In the context of DNA seuqnce analysis, these are primarily composed of the nucleotides A, T, G , and C. Finding frequently occuring k-mers and their distribution has multiple applications, such as assembling DNA sequences into genomes, finding elements important in replication to use in heterologous gene expression, identifying species in metagenomics analysis etc. It is thus an exercise encountered rather frequently in the bioinformatics realm.

### <B> First approach
A simple approach would be to identify all the k-mers that occur in a string and then compute how many times each k-mer appears in the string as demonstrated below.

In [22]:
# We use an inbuilt profiler for to check where the bottlenecks might be
%load_ext line_profiler

In [23]:
import cProfile

In [24]:
def FrequentWords(Text, k):
    '''
    This function takes a string Text and a number k as input to find the 
    most frequently occuring substring of length k within Text.
    It uses a sliding window approach to scan the Text and note each pattern of length k 
    and keeps count of every time the pattern is found subsequently in the string.
    It returns the most frequently occuring k-mer string as output.
    '''
    FrequentPatterns = []
    n = len(Text)
    Count = [0] * (n-k+1)
    
    for i in range(n-k+1):
        Pattern = Text[i: i+k]
        Count[i] = PatternCount(Text, Pattern)
    m = max(Count)
    
    for i in range(n-k+1):
        if Count[i] == m:
            FrequentPatterns.append(Text[i:i+k])
    
    #remove duplicates from FrequentPatterns
    FrequentPatterns = list(set(FrequentPatterns))
        
    return FrequentPatterns


def PatternCount(Text, Pattern):
    '''
    This function counts how many times the smaller substring Pattern occurs in the longer string Text 
    and returns the number count.
    '''
    count = 0
    for i in range(len(Text)-len(Pattern)):
        if Text[i:i+len(Pattern)] == Pattern:
            count += 1
            
    return count

#### <B> 1. First example
As an example, we use a short string below to find the most frequently occuring pemtamers 

In [25]:
text = "ACAACTATGCATACTATCGGGAACTATCCT"
k = 5
print(FrequentWords(text,k))

['ACTAT']


#### <B> 2. Second example
In our second example, we use a slightly longer DNA sequence to find a longer k-mer

In [26]:
T = "TCTTTGGAAGGATTTTTAGGATTTTTAGCGGAAGAGCGGAAGTAGATTGGTAGGATTTTTTCTTTGGAAGCGGAAGTCTTTGGATAGATTGGTAGGATTTTTAGCGGAAGGCAGATAGCAGATATCTTTGGATAGATTGGTGCAGATATAGATTGGTGCAGATATCTTTGGAGCAGATAGCAGATATAGATTGGTTAGATTGGTGCAGATAAGGATTTTTTAGATTGGTTAGATTGGTAGGATTTTTGCAGATAAGGATTTTTTAGATTGGTTAGATTGGTGCAGATAGCAGATAAGCGGAAGGCAGATAGCAGATAAGGATTTTTTAGATTGGTAGCGGAAGTAGATTGGTAGCGGAAGAGCGGAAGTAGATTGGTAGCGGAAGTAGATTGGTGCAGATAAGGATTTTTAGGATTTTTTCTTTGGATAGATTGGTTCTTTGGAAGGATTTTTAGGATTTTTTAGATTGGTGCAGATATAGATTGGTTCTTTGGAAGCGGAAGAGCGGAAGAGCGGAAGTAGATTGGTTCTTTGGAAGCGGAAGAGCGGAAGTCTTTGGAAGCGGAAGTAGATTGGTAGGATTTTTAGGATTTTTTCTTTGGAGCAGATATCTTTGGAAGGATTTTTGCAGATATAGATTGGTGCAGATATCTTTGGAAGGATTTTTAGCGGAAGTCTTTGGAGCAGATAAGGATTTTTTCTTTGGAAGGATTTTTTAGATTGGTAGCGGAAGTCTTTGGATCTTTGGAAGCGGAAGTCTTTGGATAGATTGGTTAGATTGGTAGCGGAAGAGGATTTTTGCAGATATCTTTGGATAGATTGGTAGCGGAAGGCAGATATCTTTGGAAGCGGAAGTAGATTGGTAGGATTTTTTCTTTGGAGCAGATA"
k = 13
print(FrequentWords(T,k))

['TAGATTGGTGCAG', 'GCGGAAGTAGATT', 'CGGAAGTAGATTG', 'ATTGGTGCAGATA', 'AGATTGGTGCAGA', 'GGAAGTAGATTGG', 'AGCGGAAGTAGAT', 'GATTGGTGCAGAT', 'GAAGTAGATTGGT']


As you can see, our function actually returns multiple k-mers corresponding to the highest count.

#### <B> 3. Profiling the code

In [27]:
%lprun -f FrequentWords FrequentWords(T, k)

Timer unit: 1e-09 s

Total time: 0.105749 s
File: /tmp/ipykernel_6553/3726784926.py
Function: FrequentWords at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def FrequentWords(Text, k):
     2                                               '''
     3                                               This function takes a string Text and a number k as input to find the 
     4                                               most frequently occuring substring of length k within Text.
     5                                               It uses a sliding window approach to scan the Text and note each pattern of length k 
     6                                               and keeps count of every time the pattern is found subsequently in the string.
     7                                               It returns the most frequently occuring k-mer string as output.
     8                                               '''
  

#### <B> Discussion
Although this approach works alright for such a small string of DNA, it is most likely to get clunky and slow on strings of much larger size which one is more likely to encounter while analyzing real world problems. <br>
Each call to the <I>PatternCount</I> checks if the pattern matches the substring at position 0, 1 and so on till the end of the string Text. This results in $|Text|-k+1$ comparisons for every k-mer in the string, and the higher level function <I> FrequentWords </I> calls this function for each k-mer of the Text, resulting in $(|Text|-k+1).(|Text|-k+1).k$ steps. This means that the complexity of the function <I>FrequentWords</I> becomes $O(|Text|^{2}.k)$. You can actually observe this phenomenon in our code profile above, where the calls to the function <I>PatternCount</I> are indeed very high and this step takes up 99.6% of the computation time. <br>

Now imagine working with a DNA string millions of base pairs in length, and the magnitude of the complexity quickly becomes very daunting.
We must thus look for a better solution!

### <B> A Different Approach to the same problem

We can tackle the same problem by using dictionaries to help keep count of the number of times each k-mer is encountered in a text string, or in short, generate a frequency table for all unique kmers, and then call out the ones with maximum number of occurences. This way, we only slide the window once down the entire length of the text bringing the complexity down to $O(|Text|)$.

In [32]:
def BetterFrequentWords(Text, k):
    '''
    This function takes a string Text and an integer k as input 
    and returns the most frequently occuring substring k.
    It uses the FrequencyTable and MaxMap subroutines to generate 
    a dictionary of k-mers and their count, and to find the maximum value.
    It returns a list of k-mers corresponding to the maximum value.
    '''
    FrequentPatterns = []
    freqMap = FrequencyTable(Text, k)
    maxval = MaxMap(freqMap)
    for k,v in freqMap.items():
        if v == maxval:
            FrequentPatterns.append(k)
            
    return FrequentPatterns
		
def MaxMap(freqmap):
    '''
    Takes a dictionary freqmap as input and returns 
    the maximum value in the dictionary.
    '''
    maxvalue = max(freqmap.values())
    return maxvalue
	

def FrequencyTable(Text, k):
    '''
    It takes a string Text and an integer k as input.
    Generates a dictionary using k-mers as keys and 
    increments the value every time the k-mer is present in the string Text.
    Returns this dictionary.
    '''
    freqMap = {}
    n = len(Text)
    for i in range(n-k):
        pattern = Text[i:i+k]
        if pattern in freqMap.keys():
            freqMap[pattern] += 1
        else:
            freqMap[pattern] = 1
    return freqMap

#### <B> 1. First example
We use the same example as before to test that our code actually works

In [33]:
text = "ACAACTATGCATACTATCGGGAACTATCCT"
k = 5
print(BetterFrequentWords(text,k))

['ACTAT']


#### <B> 2. Second Example
Then we look at the other example with longer text and k-mer

In [34]:
T = "TCTTTGGAAGGATTTTTAGGATTTTTAGCGGAAGAGCGGAAGTAGATTGGTAGGATTTTTTCTTTGGAAGCGGAAGTCTTTGGATAGATTGGTAGGATTTTTAGCGGAAGGCAGATAGCAGATATCTTTGGATAGATTGGTGCAGATATAGATTGGTGCAGATATCTTTGGAGCAGATAGCAGATATAGATTGGTTAGATTGGTGCAGATAAGGATTTTTTAGATTGGTTAGATTGGTAGGATTTTTGCAGATAAGGATTTTTTAGATTGGTTAGATTGGTGCAGATAGCAGATAAGCGGAAGGCAGATAGCAGATAAGGATTTTTTAGATTGGTAGCGGAAGTAGATTGGTAGCGGAAGAGCGGAAGTAGATTGGTAGCGGAAGTAGATTGGTGCAGATAAGGATTTTTAGGATTTTTTCTTTGGATAGATTGGTTCTTTGGAAGGATTTTTAGGATTTTTTAGATTGGTGCAGATATAGATTGGTTCTTTGGAAGCGGAAGAGCGGAAGAGCGGAAGTAGATTGGTTCTTTGGAAGCGGAAGAGCGGAAGTCTTTGGAAGCGGAAGTAGATTGGTAGGATTTTTAGGATTTTTTCTTTGGAGCAGATATCTTTGGAAGGATTTTTGCAGATATAGATTGGTGCAGATATCTTTGGAAGGATTTTTAGCGGAAGTCTTTGGAGCAGATAAGGATTTTTTCTTTGGAAGGATTTTTTAGATTGGTAGCGGAAGTCTTTGGATCTTTGGAAGCGGAAGTCTTTGGATAGATTGGTTAGATTGGTAGCGGAAGAGGATTTTTGCAGATATCTTTGGATAGATTGGTAGCGGAAGGCAGATATCTTTGGAAGCGGAAGTAGATTGGTAGGATTTTTTCTTTGGAGCAGATA"
k = 13
print(BetterFrequentWords(T,k))

['AGCGGAAGTAGAT', 'GCGGAAGTAGATT', 'CGGAAGTAGATTG', 'GGAAGTAGATTGG', 'GAAGTAGATTGGT', 'TAGATTGGTGCAG', 'AGATTGGTGCAGA', 'GATTGGTGCAGAT', 'ATTGGTGCAGATA']


#### <B> 3. Profiling
No we profile the code once again to see if we got id of the bottle neck and compare the efficiency

In [35]:
%lprun -f BetterFrequentWords BetterFrequentWords(T, k)

Timer unit: 1e-09 s

Total time: 0.00031193 s
File: /tmp/ipykernel_6553/3418861718.py
Function: BetterFrequentWords at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def BetterFrequentWords(Text, k):
     2                                               '''
     3                                               This function takes a string Text and an integer k as input 
     4                                               and returns the most frequently occuring substring k.
     5                                               It uses the FrequencyTable and MaxMap subroutines to generate 
     6                                               a dictionary of k-mers and their count, and to find the maximum value.
     7                                               It returns a list of k-mers corresponding to the maximum value.
     8                                               '''
     9         1        463.0    46

#### <B> Discussion
Look at that! We have already made this function about 300 times faster than the first one. And as you can see, the number of calls made to the subroutine <I>FrequencyTable</I> are much lower than the calls to <I>PatternCount</I> in the previous function. <br>
Now if you wanted to make this even faster, you know that the answer lies in making the <I>FrequencyTable</I> function more efficient.